In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread, pprint
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('bplinebot-3ccea59ad6d6.json', scope)
client = gspread.authorize(creds)
pp = pprint.PrettyPrinter()
def get_client():
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('bplinebot-3ccea59ad6d6.json', scope)
    client = gspread.authorize(creds)
    return client

In [16]:
#sheet = client.open('AbbokIncomeAssesmentV03').get_worksheet(2)
spread = client.open('userCheckin')

In [17]:
sheet = spread.worksheet('log')

In [18]:
list_of_lists = sheet.get_all_values()

In [30]:
df = pd.DataFrame(list_of_lists[1:], columns = list_of_lists[0])
df = df[pd.to_datetime(df['DATE']).dt.month == datetime.today().month]#datetime.today().month
df.head(5)

,ID,DATE,TIME,USER_ID,TYPE
438,439,2019/09/02,09:29 AM,2,1
439,440,2019/09/02,09:41 AM,3,1
440,441,2019/09/02,09:42 AM,7,1
441,442,2019/09/02,10:24 AM,8,1
442,443,2019/09/02,12:20 PM,1,1


In [31]:
df['TIME'] = pd.to_datetime(df['TIME']).values.astype(np.int64)
df['DATE'] = pd.to_datetime(df['DATE']).dt.date
df['USER_ID'] = df['USER_ID'].astype(int)
df['TYPE'] = df['TYPE'].astype(int)
df = df.set_index('ID')
df.shape

(88, 4)

In [32]:
df2 = pd.DataFrame(df.groupby(['USER_ID', 'TYPE'])['TIME'].mean())

In [33]:
df2['INOUT'] = pd.to_datetime(df2['TIME']).dt.time
df2

TIME            INOUT
USER_ID TYPE                                      
1       0     1568055080000000000         18:51:20
        1     1568040984000000000         14:56:24
2       0     1568037612000000000         14:00:12
        1     1568022500000000000         09:48:20
3       0     1568042920000000000         15:28:40
        1     1568034918000000000         13:15:18
4       0     1568060140000000000         20:15:40
        1     1568019380000000000         08:56:20
7       0     1568052168000000000         18:02:48
        1     1568022490000000000         09:48:10
8       0     1568051290000000000         17:48:10
        1     1568023671428571392  10:07:51.428571
9       0     1568060835000000000         20:27:15
        1     1568022936000000000         09:55:36

In [24]:
df2 = df2.reset_index()

In [25]:
w_out = df2[df2['TYPE'] == 0].reset_index().drop(["index","TYPE"], axis=1)
w_in = df2[df2['TYPE'] == 1].reset_index().drop(["index","TYPE"], axis=1)

In [26]:
w_total = w_out.copy()
w_total['TIME'] = w_out['TIME'] - w_in['TIME']
w_total

,USER_ID,TIME,INOUT
0,1,14096000000000,18:51:20
1,2,15112000000000,14:00:12
2,3,8002000000000,15:28:40
3,4,40760000000000,20:15:40
4,7,29678000000000,18:02:48
5,8,27618571428608,17:48:10
6,9,37899000000000,20:27:15


In [27]:
w_total['TIME'] = pd.to_datetime(w_total['TIME']).dt.time

In [28]:
w_total

,USER_ID,TIME,INOUT
0,1,03:54:56,18:51:20
1,2,04:11:52,14:00:12
2,3,02:13:22,15:28:40
3,4,11:19:20,20:15:40
4,7,08:14:38,18:02:48
5,8,07:40:18.571428,17:48:10
6,9,10:31:39,20:27:15


In [29]:
w_total[w_total['USER_ID'] == 1]['TIME'].values[0].strftime("%H:%M:%S")

'03:54:56'